# 2.1. Getting SQL Data into a DataFrame

Running the kind of queries we did in the last section is great for exploring and getting to know the data. It is also useful if you just need a one time quick answer. But more often, what you want to do is to get the data from a database into a DataFrame so that you can visualize it, combine it with other data, or do further analysis. For this exercise, our goal is to create a Google Map with markers for all of the stations in the database.

Pandas provides us with a great way to use our SQL knowledge to create a DataFrame with the results. pd.read_sql allows us to create a DataFrame by passing in an SQL query as a string, along with a URL to connect to the database. Let’s try that on our stations table now.

In [1]:
#connecting to the database

%load_ext sql
%sql sqlite:////Users/kkoum/Documents/database.db

'Connected: @/Users/kkoum/Documents/database.db'

In [2]:
#importing the appropriate packages

import pandas as pd
from ipyleaflet import Map, CircleMarker, AntPath, Heatmap #for interactive maps in the Jupyter notebook
import os #for using operating system dependent functionality
from zipfile import ZipFile #for working directly with ZIP files
import altair #for declarative statistical visualization

In [3]:
stations = pd.read_sql_query('SELECT * FROM stations','sqlite:////Users/kkoum/Documents/database.db')
stations.head()

,station_id,name,capacity,status,latitude,longitude
0,31000,Eads St & 15th St S,15,OPEN,38.858971,-77.053230
1,31001,18th St & S Eads St,11,OPEN,38.857250,-77.053320
2,31002,Crystal Dr & 20th St S,17,OPEN,38.856425,-77.049232
3,31003,Crystal Dr & 15th St S,10,OPEN,38.860170,-77.049593
4,31004,Aurora Hills Cmty Ctr / 18th St & S Hayes St,11,OPEN,38.857866,-77.059490


We can use any SQL query we want and pandas is ‘smart’ enough to look at the results and create columns that match the columns in the database. If we know we are going to have columns containing dates it's a good idea to use the parse_dates parameter when we call read_sql_query. This will ensure that our date columns are of the correct type without us having to convert them later.

Let's create a DataFrame of the first 10 rides from the trip_data table sorted by start_date.

In [4]:
trip_data = pd.read_sql_query('SELECT * from trip_data ORDER BY start_date LIMIT 10','sqlite:////Users/kkoum/Documents/database.db', parse_dates = ['start_date', 'end_date'])
trip_data.head()

,start_date,end_date,duration,start_station_id,start_station,end_station_id,end_station,bike_number,member_type
0,2012-01-01 00:04:00,2012-01-01 00:11:56,475,31245,7th & R St NW / Shaw Library,31109,7th & T St NW,W01412,Member
1,2012-01-01 00:10:05,2012-01-01 00:29:28,1162,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00524,Casual
2,2012-01-01 00:10:23,2012-01-01 00:29:28,1145,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00235,Member
3,2012-01-01 00:15:41,2012-01-01 00:23:46,485,31101,14th & V St NW,31602,Park Rd & Holmead Pl NW,W00864,Member
4,2012-01-01 00:15:42,2012-01-01 00:23:34,471,31102,11th & Kenyon St NW,31109,7th & T St NW,W00995,Member


Then add a column called calc_duration where you calculate the duration of the ride by subtracting the start date from the end date.

In [5]:
trip_data['calc_duration'] = trip_data['end_date'] - trip_data['start_date']
trip_data.head()

,start_date,end_date,duration,start_station_id,start_station,end_station_id,end_station,bike_number,member_type,calc_duration
0,2012-01-01 00:04:00,2012-01-01 00:11:56,475,31245,7th & R St NW / Shaw Library,31109,7th & T St NW,W01412,Member,00:07:56
1,2012-01-01 00:10:05,2012-01-01 00:29:28,1162,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00524,Casual,00:19:23
2,2012-01-01 00:10:23,2012-01-01 00:29:28,1145,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00235,Member,00:19:05
3,2012-01-01 00:15:41,2012-01-01 00:23:46,485,31101,14th & V St NW,31602,Park Rd & Holmead Pl NW,W00864,Member,00:08:05
4,2012-01-01 00:15:42,2012-01-01 00:23:34,471,31102,11th & Kenyon St NW,31109,7th & T St NW,W00995,Member,00:07:52


What is the type of the calc_duration field that you just computed?

In [6]:
trip_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
start_date          10 non-null datetime64[ns]
end_date            10 non-null datetime64[ns]
duration            10 non-null int64
start_station_id    10 non-null int64
start_station       10 non-null object
end_station_id      10 non-null int64
end_station         10 non-null object
bike_number         10 non-null object
member_type         10 non-null object
calc_duration       10 non-null timedelta64[ns]
dtypes: datetime64[ns](2), int64(3), object(4), timedelta64[ns](1)
memory usage: 928.0+ bytes


What's the value for the first row of calc_duration?

In [7]:
trip_data['calc_duration'].head(1)

0   00:07:56
Name: calc_duration, dtype: timedelta64[ns]

What about the last row?

In [8]:
trip_data['calc_duration'].tail(1)

9   00:15:14
Name: calc_duration, dtype: timedelta64[ns]

# 2.2. Mapping Bike Stations

Now let's have some fun. We are going to map the location of all of the bike stations on a google map! Later you can explore all kinds of ways to visualize rides and ride frequencies between starting and ending points and many more things.

To map our stations we will use the ipyleaflet module. The easiest way to install ipyleaflet is by using anaconda, which you probably already have on your computer. For the ones not using anaconda the process is the same, but with different commands. Supposing you have anaconda, complete the following steps:

* Οpen the anaconda prompt
* To install ipyleaflet, use the command:

  conda install -c conda-forge ipyleaflet


* To make ipyleaflet work on Jupyterlab, you also need to install the jupyter-leaflet extension for Jupyterlab. But in order to do that, you must have nodejs and npm on your computer. To install them, type:

  conda install -c conda-forge nodejs


* Now you are ready to install the the jupyter-leaflet extension:

  jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet


* Remember to restart the kernel, in order for the above changes to take place!

After completing all of the prerequisites, our next task is to create a list of all the latitude and longitude values for each station. The challenge here is that the leaflet interface wants to get a list that looks like this: [(lat1, long1), (lat2, long2), ...]. That means we need to combine the two columns from our DataFrame into a list of tuples. You could, of course, do this with a for loop, iterating over all the rows, but there is a much easier way.

Suppose we have two lists:

list1 = ['a', 'b', 'c']

list2 = [1, 2, 3]

We can combine these into [('a', 1), ('b', 2), ('c', 3)], using Python’s zip function, like this:

zip(list1, list2)

The zip function takes any number of iterable objects and “zips” them together, taking the first element from each list and making a tuple, then the second etc. Zip returns an iterator, which is basically an object that can be iterated upon. To convert an iterator into a list, we just have to put it inside a list fuction, like this:

list(zip(list1, list2))

In [9]:
locations = list(zip(stations.latitude, stations.longitude))

stations_map_center = (38.9072, -77.0369)

stations_map = Map(center = stations_map_center, zoom = 12)
for i in locations:
    marker = CircleMarker(location = i, radius=2)
    stations_map.add_layer(marker)

stations_map

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## 2.2.1. Practice and Further Exploration

The map seems a little busy with all those markers. Redo the map so that it only shows the 20 busiest stations. That is the 20 stations with the highest starting counts.

In [10]:
trip_data = pd.read_csv(r'C:\Users\kkoum\Documents\1. University\Business study\Dataset\trip_data_2012.csv')
stations = pd.read_csv(r'C:\Users\kkoum\Documents\1. University\Business study\Dataset\stations.csv')

In [11]:
trips_stations = pd.merge(trip_data, stations, left_on = 'start_station_id', right_on = 'station_id', how = 'left')
trips_stations.head()

,start_date,end_date,duration,start_station_id,start_station,end_station_id,end_station,bike_number,member_type,station_id,name,capacity,status,latitude,longitude
0,2012-01-01 00:04:00,2012-01-01 00:11:56,475,31245,7th & R St NW / Shaw Library,31109,7th & T St NW,W01412,Member,31245.0,7th & R St NW / Shaw Library,15.0,OPEN,38.912719,-77.022155
1,2012-01-01 00:10:05,2012-01-01 00:29:28,1162,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00524,Casual,31400.0,Georgia & New Hampshire Ave NW,19.0,OPEN,38.936684,-77.024181
2,2012-01-01 00:10:23,2012-01-01 00:29:28,1145,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00235,Member,31400.0,Georgia & New Hampshire Ave NW,19.0,OPEN,38.936684,-77.024181
3,2012-01-01 00:15:41,2012-01-01 00:23:46,485,31101,14th & V St NW,31602,Park Rd & Holmead Pl NW,W00864,Member,31101.0,14th & V St NW,31.0,OPEN,38.917931,-77.032112
4,2012-01-01 00:15:42,2012-01-01 00:23:34,471,31102,11th & Kenyon St NW,31109,7th & T St NW,W00995,Member,31102.0,11th & Kenyon St NW,27.0,CLOSED,38.929464,-77.027822


In [12]:
trips_stations['rides'] = trips_stations.groupby('station_id')['start_date'].transform('count')
trips_stations = trips_stations.drop_duplicates(subset = "station_id")
trips_stations = trips_stations.sort_values(by = 'rides', ascending = False)
trips_stations.head()

,start_date,end_date,duration,start_station_id,start_station,end_station_id,end_station,bike_number,member_type,station_id,name,capacity,status,latitude,longitude,rides
27,2012-01-01 00:48:55,2012-01-01 01:04:26,931,31200,Massachusetts Ave & Dupont Circle NW,31604,3rd & H St NW,W01213,Member,31200.0,Massachusetts Ave & Dupont Circle NW,28.0,OPEN,38.91010,-77.044400,58069.0
51,2012-01-01 01:08:06,2012-01-01 01:14:07,360,31623,Columbus Circle / Union Station,31614,11th & H St NE,W00021,Member,31623.0,Columbus Circle / Union Station,55.0,OPEN,38.89696,-77.004930,45692.0
25,2012-01-01 00:45:58,2012-01-01 00:54:18,499,31201,15th & P St NW,31229,New Hampshire Ave & T St NW,W00996,Member,31201.0,15th & P St NW,19.0,OPEN,38.90985,-77.034438,40823.0
19,2012-01-01 00:36:11,2012-01-01 00:52:08,956,31214,17th & Corcoran St NW,31214,17th & Corcoran St NW,W00174,Member,31214.0,17th & Corcoran St NW,19.0,OPEN,38.91210,-77.038700,35342.0
89,2012-01-01 01:32:58,2012-01-01 01:53:24,1226,31241,Thomas Circle,31505,Eckington Pl & Q St NE,W00137,Member,31241.0,Thomas Circle,33.0,OPEN,38.90590,-77.032500,33875.0


In [13]:
locations = list(zip(trips_stations[:20].latitude, trips_stations[:20].longitude))
stations_map_center = (38.9072, -77.0369)

stations_map = Map(center = stations_map_center, zoom = 12)
for i in locations:
    marker = CircleMarker(location = i, radius=2)
    stations_map.add_layer(marker)

stations_map

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

It would also be interesting to add the 20 stations with the largest ending counts. Can you add those in a different color?

In [14]:
trips_stations = pd.merge(trip_data, stations, left_on = 'end_station_id', right_on = 'station_id', how = 'left')

trips_stations['rides'] = trips_stations.groupby('station_id')['start_date'].transform('count')
trips_stations = trips_stations.drop_duplicates(subset = "station_id")
trips_stations = trips_stations.sort_values(by = 'rides', ascending = False)

locations = list(zip(trips_stations[:20].latitude, trips_stations[:20].longitude))
stations_map_center = (38.9072, -77.0369)

for i in locations:
    marker = CircleMarker(location = i, radius=2, color = 'red')
    stations_map.add_layer(marker)

stations_map

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

Investigate the interface to see if there is a way for you to color code the markers based on the number of rides originating from that station. Show the 50 most popular stations using five different colors. 1 color for the top 10, another for the next 10 and so on.

In [15]:
trips_stations = pd.merge(trip_data, stations, left_on = 'start_station_id', right_on = 'station_id', how = 'left')

trips_stations['rides'] = trips_stations.groupby('station_id')['start_date'].transform('count')
trips_stations = trips_stations.drop_duplicates(subset = "station_id")
trips_stations = trips_stations.sort_values(by = 'rides', ascending = False)

locations = list(zip(trips_stations[:50].latitude, trips_stations[:50].longitude))
stations_map_center = (38.9072, -77.0369)

stations_map = Map(center = stations_map_center, zoom = 12)
for i in locations[:10]:
    marker = CircleMarker(location = i, radius=2, color = 'blue')
    stations_map.add_layer(marker)

for i in locations[10:20]:
    marker = CircleMarker(location = i, radius=2, color = 'red')
    stations_map.add_layer(marker)

for i in locations[20:30]:
    marker = CircleMarker(location = i, radius=2, color = 'yellow')
    stations_map.add_layer(marker)
    
for i in locations[30:40]:
    marker = CircleMarker(location = i, radius=2, color = 'green')
    stations_map.add_layer(marker)
    
for i in locations[40:50]:
    marker = CircleMarker(location = i, radius=2, color = 'orange')
    stations_map.add_layer(marker)
stations_map

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

Here is a real challenge for you: Pick a station and then follow the rentals, but only map the stations where a bike ends up back at your original starting point. In other words, only for the bikes that started at station A, go to station B, then D, then pause for a while, then on to station X and then finally back to A. Others may take a different route. If you can do this you can investigate the AntPath layer that lets you add a route to the map! Then you can show the routes that the various bikes took to make their way back to the starting point. You may want to limit the time on this to one day or a week.

In [16]:
trips_stations = pd.merge(trip_data, stations, left_on = 'start_station_id', right_on = 'station_id', how = 'left')
trips_stations = trips_stations[trips_stations['start_date'] < '2012-01-05']
trips_stations.head()

,start_date,end_date,duration,start_station_id,start_station,end_station_id,end_station,bike_number,member_type,station_id,name,capacity,status,latitude,longitude
0,2012-01-01 00:04:00,2012-01-01 00:11:56,475,31245,7th & R St NW / Shaw Library,31109,7th & T St NW,W01412,Member,31245.0,7th & R St NW / Shaw Library,15.0,OPEN,38.912719,-77.022155
1,2012-01-01 00:10:05,2012-01-01 00:29:28,1162,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00524,Casual,31400.0,Georgia & New Hampshire Ave NW,19.0,OPEN,38.936684,-77.024181
2,2012-01-01 00:10:23,2012-01-01 00:29:28,1145,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00235,Member,31400.0,Georgia & New Hampshire Ave NW,19.0,OPEN,38.936684,-77.024181
3,2012-01-01 00:15:41,2012-01-01 00:23:46,485,31101,14th & V St NW,31602,Park Rd & Holmead Pl NW,W00864,Member,31101.0,14th & V St NW,31.0,OPEN,38.917931,-77.032112
4,2012-01-01 00:15:42,2012-01-01 00:23:34,471,31102,11th & Kenyon St NW,31109,7th & T St NW,W00995,Member,31102.0,11th & Kenyon St NW,27.0,CLOSED,38.929464,-77.027822


In [17]:
picked_station_id = 31623

picked_bikes = trips_stations[trips_stations['start_station_id'] == picked_station_id]
picked_bikes = picked_bikes.groupby('bike_number')[['start_date']].count().reset_index()
picked_bikes.columns = ['bike_number', 'rides']
picked_bikes = picked_bikes[picked_bikes['rides'] > 2]
picked_bikes.head()

,bike_number,rides
35,W00232,3
38,W00241,4
40,W00265,3
43,W00302,3
65,W00432,4


In [18]:
picked_bikes_trips = trips_stations[trips_stations['start_date'] == ''] #empty DataFrame with all the columns

for i in picked_bikes.bike_number:
    picked_bikes_trips = picked_bikes_trips.append(trips_stations[trips_stations['bike_number'] == i])
picked_bikes_trips = picked_bikes_trips.reset_index()
picked_bikes_trips.head()

,index,start_date,end_date,duration,start_station_id,start_station,end_station_id,end_station,bike_number,member_type,station_id,name,capacity,status,latitude,longitude
0,35,2012-01-01 00:54:03,2012-01-01 01:01:44,460,31606,Potomac & Pennsylvania Ave SE,31622,13th & D St NE,W00232,Member,31606.0,Potomac & Pennsylvania Ave SE,15.0,OPEN,38.880300,-76.986200
1,2391,2012-01-02 08:50:02,2012-01-02 08:55:08,305,31619,Lincoln Park / 13th & East Capitol St NE,31605,3rd & D St SE,W00232,Member,31619.0,Lincoln Park / 13th & East Capitol St NE,25.0,OPEN,38.890461,-76.988355
2,2574,2012-01-02 10:57:16,2012-01-02 11:06:05,529,31605,3rd & D St SE,31623,Columbus Circle / Union Station,W00232,Member,31605.0,3rd & D St SE,11.0,OPEN,38.885100,-77.002300
3,2604,2012-01-02 11:09:16,2012-01-02 11:18:42,565,31623,Columbus Circle / Union Station,31626,15th St & Massachusetts Ave SE,W00232,Member,31623.0,Columbus Circle / Union Station,55.0,OPEN,38.896960,-77.004930
4,4042,2012-01-02 20:46:27,2012-01-02 20:50:34,246,31626,15th St & Massachusetts Ave SE,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,W00232,Member,31626.0,15th St & Massachusetts Ave SE,15.0,OPEN,38.887320,-76.983569


In [19]:
start_indexes = picked_bikes_trips[(picked_bikes_trips.start_station_id == picked_station_id)]
start_indexes = start_indexes.index.to_list()
start_indexes[ : 5]

[3, 8, 10, 14, 16]

In [20]:
round_trips = picked_bikes_trips[picked_bikes_trips['start_date'] == '']
round_trips_temp = picked_bikes_trips[picked_bikes_trips['start_date'] == '']

for i in start_indexes[ : len(start_indexes) - 1]:
    flag = True
    for j in range(i, start_indexes[start_indexes.index(i) + 1]):
        round_trips_temp = round_trips_temp.append(picked_bikes_trips.iloc[[j]])
        if picked_bikes_trips.end_station_id[j] != picked_bikes_trips.start_station_id[j + 1]:
            flag = False
    if len(round_trips_temp) > 2:
        if flag == True:
            round_trips = round_trips.append(round_trips_temp)
            round_trips = round_trips.append(round_trips_temp.iloc[0]) #To close the polygon on the map
    round_trips_temp = picked_bikes_trips[picked_bikes_trips['start_date'] == '']
round_trips.head()

,index,start_date,end_date,duration,start_station_id,start_station,end_station_id,end_station,bike_number,member_type,station_id,name,capacity,status,latitude,longitude
3,2604,2012-01-02 11:09:16,2012-01-02 11:18:42,565,31623,Columbus Circle / Union Station,31626,15th St & Massachusetts Ave SE,W00232,Member,31623.0,Columbus Circle / Union Station,55.0,OPEN,38.896960,-77.004930
4,4042,2012-01-02 20:46:27,2012-01-02 20:50:34,246,31626,15th St & Massachusetts Ave SE,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,W00232,Member,31626.0,15th St & Massachusetts Ave SE,15.0,OPEN,38.887320,-76.983569
5,4082,2012-01-02 21:11:55,2012-01-02 21:14:41,166,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31607,14th & D St SE,W00232,Member,31613.0,Eastern Market Metro / Pennsylvania Ave & 7th ...,19.0,OPEN,38.884000,-76.995397
6,4109,2012-01-02 21:41:29,2012-01-02 21:47:39,370,31607,14th & D St SE,31610,Eastern Market / 7th & North Carolina Ave SE,W00232,Member,31607.0,13th & E St SE,15.0,OPEN,38.882915,-76.987907
7,4312,2012-01-03 06:56:32,2012-01-03 07:06:09,576,31610,Eastern Market / 7th & North Carolina Ave SE,31623,Columbus Circle / Union Station,W00232,Member,31610.0,Eastern Market / 7th & North Carolina Ave SE,15.0,OPEN,38.886952,-76.996806


In [21]:
station_latitude = trips_stations.loc[trips_stations.station_id == picked_station_id, 'latitude'].iloc[0]
station_longitude = trips_stations.loc[trips_stations.station_id == picked_station_id, 'longitude'].iloc[0]

stations_map_center = (station_latitude, station_longitude)
stations_map = Map(center = stations_map_center, zoom = 12)

marker = CircleMarker(location = (station_latitude, station_longitude), radius=3, color = 'red')
stations_map.add_layer(marker)

locations = list(zip(round_trips.latitude, round_trips.longitude))
ant_path = AntPath(locations = locations, delay = 1000)
stations_map.add_layer(ant_path)

stations_map

Map(center=[38.89696, -77.00493], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

Another Challenge: The ipyleaflet interface allows you to add a heatmap layer. This sounds like it could a very interesting way to overlay the popularity of different stations and routes on the map. Investigate this layer and what the data should look like, then see if you can find a way to get the data into the appropriate form to make a heatmap.

In [22]:
trips_stations = pd.merge(trip_data, stations, left_on = 'start_station_id', right_on = 'station_id', how = 'left')
station_heat = trips_stations.groupby('station_id')['start_date'].count().reset_index()
station_heat.columns = ['station_id', 'heat']
station_heat.head()

,station_id,heat
0,31000.0,1353
1,31001.0,3393
2,31002.0,4210
3,31003.0,2443
4,31004.0,1623


In [23]:
station_heat = pd.merge(stations, station_heat, on = 'station_id')
station_heat.head()

,station_id,name,capacity,status,latitude,longitude,heat
0,31000,Eads St & 15th St S,15,OPEN,38.858971,-77.053230,1353
1,31001,18th St & S Eads St,11,OPEN,38.857250,-77.053320,3393
2,31002,Crystal Dr & 20th St S,17,OPEN,38.856425,-77.049232,4210
3,31003,Crystal Dr & 15th St S,10,OPEN,38.860170,-77.049593,2443
4,31004,Aurora Hills Cmty Ctr / 18th St & S Hayes St,11,OPEN,38.857866,-77.059490,1623


In [24]:
loc = list(zip(station_heat.latitude, station_heat.longitude, station_heat.heat))
stations_map_center = (38.9072, -77.0369)

stations_map = Map(center = stations_map_center, zoom = 12)

heatmap = Heatmap(locations = loc, radius = 17)
stations_map.add_layer(heatmap);

stations_map

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

# 2.3. Working with ZIP Files

We have access to the data of all the bike trips from 2011 to 2020 in ZIP format. As is usually the case with large files, our CSVs are available in compressed format (ZIP).

Python allows us to work with the ZIP files directly so we don’t have to manually uncompress and recompress the files just to save disk space. Let's create a DataFrame of all the bike share data, with the assumption that you have downloaded all the files you need into a folder, and have no other files in that folder.

In [25]:
zipfiles = os.listdir('C:/Users/kkoum/Documents/1. University/Business study/Dataset/Zipped')
zipfiles

['trip_data_2011.zip',
 'trip_data_2012.zip',
 'trip_data_2013.zip',
 'trip_data_2014.zip',
 'trip_data_2015.zip',
 'trip_data_2016.zip',
 'trip_data_2017.zip',
 'trip_data_2018.zip',
 'trip_data_2019.zip']

In [26]:
trip_data_temp = []

for f in zipfiles:
    with ZipFile(f'C:/Users/kkoum/Documents/1. University/Business study/Dataset/Zipped/{f}') as myzip:
        for zf in myzip.namelist():
            with myzip.open(zf) as ridedata:
                t = pd.read_csv(ridedata, parse_dates=['start_date', 'end_date'])
                trip_data_temp.append(t)

trip_data = pd.concat(trip_data_temp)
trip_data

,start_date,end_date,duration,start_station_id,start_station,end_station_id,end_station,bike_number,member_type
0,2011-01-01 00:01:29,2011-01-01 01:00:37,3548,31620,5th & F St NW,31620,5th & F St NW,W00247,Member
1,2011-01-01 00:02:46,2011-01-01 00:08:32,346,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual
2,2011-01-01 00:06:13,2011-01-01 00:15:36,562,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member
3,2011-01-01 00:09:21,2011-01-01 00:16:36,434,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member
4,2011-01-01 00:28:26,2011-01-01 00:32:19,233,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual
...,...,...,...,...,...,...,...,...,...
3398412,2019-12-31 23:57:36,2019-12-31 23:59:46,130,31011,Crystal Dr & 23rd St S,31009,Crystal Dr & 27th St S,W21285,Member
3398413,2019-12-31 23:57:47,2020-01-01 00:08:51,664,31125,15th & W St NW,31281,8th & O St NW,W24197,Member
3398414,2019-12-31 23:59:37,2020-01-01 00:06:06,389,31047,Braddock Rd Metro,31085,Mount Vernon Ave & E Nelson Ave,W21281,Member
3398415,2019-12-31 23:59:38,2020-01-01 00:15:40,962,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member


The os.listdir call looks at the directory and returns a list containing all the filenames of files there.

The "with ZipFile..." line opens the .zip file as an object that we can manipulate. With myzip.open, Python automatically uncompresses the file for us. That way, we can read the files like any other file.

The rest of lines simply read the uncompressed data, create a DataFrame, and store that DataFrame in a list for us to concatenate with all of the other DataFrames of the previous iterations into a single large DataFrame.

# 2.4. Visualizing Bike Share Data as a Time Series

In addition to visualizing the bike share data on a map, we can also visualize the popularity of bike share usage over time, using the data we uncompressed above and the final DataFrame. Containing over 25 million bike trips, it should have all the data to help us answer the following questions:

* Is bike sharing growing in popularity?

* Are there times of the year when bike sharing is more popular?

* Are there days of the week when bike sharing is more popular?

Creating the desired graph is very easy if we use the Pandas resample method, which works with timestamp data. This is a much nicer way to work with time series data than breaking the date into columns for year, month, day, hour, minute, etc. and then using the groupby method on a list of columns. It allows us to calculate lots of summary information on many interesting time scales.

The resample method takes several parameters, but the most important one is the one that specifies how you want it to resample. There are several options for this parameter, and here are the most common choices:

Parameter: Description

* B: Business day

* D: Calendar day

* W: Weekly

* M: Month end

* Q: Quarter end

* A: Year end

* H: Hourly

* T:  Minutely

* S: Secondly

Using a timestamp as an index gives you some additional power. For example, you can use the index to select a year: resamp['2012'] will return all rows for the year 2012, while resamp['2012-10'] will return all the rows for the month of October in 2012. This gives you incredible power and flexibility for working with date-indexed data.

First, we create a new DataFrame using the start_date as the index. This makes it easier for us to work with the resample method and prepare for graphing. Next, we call resamp.resample('D').count(). The 'D' parameter indicates that we want to aggregate the rides by day and count the number of rides. You could also use mean, min, max, median, and sum just like you would with groupby.

In [27]:
resamp = trip_data.set_index('start_date')
resamp = resamp.resample('D').count()
resamp = resamp.reset_index()
resamp.head()

,start_date,end_date,duration,start_station_id,start_station,end_station_id,end_station,bike_number,member_type
0,2011-01-01,959,959,959,959,959,959,959,959
1,2011-01-02,781,781,781,781,781,781,781,781
2,2011-01-03,1301,1301,1301,1301,1301,1301,1301,1301
3,2011-01-04,1536,1536,1536,1536,1536,1536,1536,1536
4,2011-01-05,1571,1571,1571,1571,1571,1571,1571,1571


Now that we have resampled the data per day, we can dive in and take a look at the graph of daily bike rentals.

In order to graph the data with Altair, we needed to reset the index as you saw above. The choice of y = 'duration' is arbitrary.

The python code required to draw the daily graph is as follows:

In [28]:
chart = altair.Chart(resamp, width=800).mark_line().encode(x='start_date', y='duration').interactive()
chart = chart.encode(x=altair.X('start_date', title='Day'))
chart = chart.encode(y=altair.Y('duration', title='Number of rides'))
chart

alt.Chart(...)

Experiment with some resampling and graphing of the data. Try to graph by business day and week, instead of day, to get an idea.

In [29]:
resamp = trip_data.set_index('start_date')
resamp = resamp.resample('B').count()
resamp = resamp.reset_index()

chart = altair.Chart(resamp, width=800).mark_line().encode(x='start_date', y='duration').interactive()
chart = chart.encode(x=altair.X('start_date', title='Business day'))
chart = chart.encode(y=altair.Y('duration', title='Number of rides'))
chart

alt.Chart(...)

In [30]:
resamp = trip_data.set_index('start_date')
resamp = resamp.resample('W').count()
resamp = resamp.reset_index()

chart = altair.Chart(resamp, width=800).mark_line().encode(x='start_date', y='duration').interactive()
chart = chart.encode(x=altair.X('start_date', title='Week'))
chart = chart.encode(y=altair.Y('duration', title='Number of rides'))
chart

alt.Chart(...)

How many rides occurred on June 9, 2012?

In [31]:
resamp = trip_data.set_index('start_date')
resamp = resamp.resample('D').count()
resamp.loc['2012-06-09', 'duration']

7421

For the week ending June 17, 2012 how many rentals were there?

In [32]:
resamp = trip_data.set_index('start_date')
resamp = resamp.resample('W').count()
resamp.loc['2012-06-10', 'duration']

49846

For the week ending June 17, 2012 what was the mean duration of all rides?

In [36]:
resamp = trip_data.set_index('start_date')
resamp = resamp.resample('W').mean()
resamp.loc['2012-06-10', 'duration']

1120.1694619427838

Graph the mean ride length on a daily scale. Do you notice any trends in terms of days that have longer or shorter rides? Make a graph that overlays all days of the week with just the business days.

In [37]:
resamp = trip_data.set_index('start_date')
resamp = resamp.resample('D').mean()
resamp = resamp.reset_index()

chart = altair.Chart(resamp, width=800).mark_line().encode(x='start_date', y='duration').interactive()
chart = chart.encode(x=altair.X('start_date', title='Day'))
chart = chart.encode(y=altair.Y('duration', title='Mean duration'))
chart

alt.Chart(...)

In [38]:
resamp = trip_data.set_index('start_date')
resamp = resamp.resample('B').mean()
resamp = resamp.reset_index()

chart = altair.Chart(resamp, width=800).mark_line().encode(x='start_date', y='duration').interactive()
chart = chart.encode(x=altair.X('start_date', title='Day'))
chart = chart.encode(y=altair.Y('duration', title='Mean duration'))
chart

alt.Chart(...)

# 2.5. Next Step: Predicting Rentals

Take another look at the first graph with all the bike rentals.

This graph indicates there is an overall upward trend in bike rentals in the DC area, although it looks like it might be starting to level off after the 2017 time frame. Within that overall trend of growth, we can see that there are seasonal variations as well as daily variations.

A time series graph like this is actually very similar to a graph of airline travel, stock prices, and others. In the next few sections, we are going to develop a model that will allow us to predict the number of bike rentals on a particular day. There are many ways we can do this, with one of the most common being the "linear regression" model. But first, let’s take a break for some pizza!